In [1]:
import tensorflow as tf
import keras

tf.compat.v1.disable_eager_execution()

2023-01-11 16:02:35.667566: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
tf.__version__

'2.11.0'

### 이미지 높이 400으로 거정, 비율이 맞게끔 너비 변환

In [4]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img

# 변환하려는 이미지 경로
target_image_path = './portrait.jpg'

# 스타일 이미지 경로
style_reference_image_path = './popova.jpg'

# 생성된 사진의 차원
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

### VGG19 사용

In [5]:
import numpy as np
from tensorflow.keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # ImageNet의 평균 픽셀 값을 더함
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

### VGG19 네트워크 설정

- input_tensor는 타깃이미지, 스타일이미지, 생성된 이미지를 행으로 쌓게됨
- 결국 input_tensor의 차원은 (3, 400, width, 3)이 됨

In [6]:
from tensorflow.keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))

# 생성된 이미지를 담을 플레이스홀더
combination_image = K.placeholder((1, img_height, img_width, 3))

# 세 개의 이미지를 하나의 배치로 합침
input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

# 세 이미지의 배치를 입력으로 받는 VGG 네트워크를 만듦
# 이 모델은 사전 훈련된 ImageNet 가중치를 로드함
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)
print('모델 로드 완료.')

80134624/80134624 [==============================] - 4s 0us/step


2023-01-11 16:04:13.903660: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 16:04:13.926642: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-01-11 16:04:13.956145: W tensorflow/c/c_api.cc:291] Operation '{name:'block5_conv4/kernel/Assign' id:383 op device:{requested: '', assigned: ''} def:{{{node block5_conv4/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](block5_conv4/kernel, block5_conv4/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't mod

모델 로드 완료.


### 콘텐츠 손실과 스타일 손실을 정의

In [7]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram


def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

### 두 손실 함수에 하나의 손실함수를 더 사용

- 생성된 이미지의 픽셀을 사용하여 계산하는 총 변위손실
- 이로서 생성된 이미지가 공간적인 연속성을 가지도록 도와주며 픽셀의 격자 무늬가 과도하게 나타나는것을 막아줌

In [8]:
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

- 최소화할 최종 손실 정의. 앞에서 봤던 세 개의 손실 함수의 평균
- 콘텐츠 손실은 block5_conv2 층 하나만을 사용해서 계산하고,
- 스타일 손실을 계산하기 위해서는 하위 층과 상위 층에 걸쳐 여러 층을 사용해야 함.
- 마지막으로 총 변위 손실으 추가해줌
- content_weight는 생성된 이미지에 타깃 콘텐츠가 얼마나 나타나는지를 조정함. 크기가 클수록 더 많이 나타남

In [9]:
# 층 이름과 활성화 텐서를 매핑한 딕셔너리
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# 콘텐츠 손실에 사용할 층
content_layer = 'block5_conv2'

# 스타일 손실에 사용할 층
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']

# 손실 항목의 가중치 평균에 사용할 가중치
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

# 모든 손실 요소를 더해 하나의 스칼라 변수로 손실을 정의함
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(target_image_features,
                                      combination_features)
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(style_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [10]:
# 손실에 대한 생성된 이미지의 그래디언트를 구함
grads = K.gradients(loss, combination_image)[0]

# 현재 손실과 그래디언트의 값을 추출하는 케라스 Function 객체임
fetch_loss_and_grads = K.function([combination_image], [loss, grads])


class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [11]:
from scipy.optimize import fmin_l_bfgs_b
import time

result_prefix = 'style_transfer_result'
iterations = 20

# 뉴럴 스타일 트랜스퍼의 손실을 최소화하기 위해 생성된 이미지에 대해 L-BFGS 최적화를 수행함
# 초기 값은 타깃 이미지임
# scipy.optimize.fmin_l_bfgs_b 함수가 벡터만 처리할 수 있기 때문에 이미지를 펼침
x = preprocess_image(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('반복 횟수:', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                     fprime=evaluator.grads, maxfun=20)
    print('현재 손실 값:', min_val)
    # 생성된 현재 이미지를 저장함
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    save_img(fname, img)
    end_time = time.time()
    print('저장 이미지: ', fname)
    print('%d 번째 반복 완료: %ds' % (i, end_time - start_time))

반복 횟수: 0


2023-01-11 16:04:22.893771: W tensorflow/c/c_api.cc:291] Operation '{name:'Variable/Assign' id:528 op device:{requested: '', assigned: ''} def:{{{node Variable/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](Variable, Variable/Initializer/initial_value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


현재 손실 값: 5486439400.0
저장 이미지:  style_transfer_result_at_iteration_0.png
0 번째 반복 완료: 156s
반복 횟수: 1
현재 손실 값: 1703190500.0
저장 이미지:  style_transfer_result_at_iteration_1.png
1 번째 반복 완료: 184s
반복 횟수: 2
현재 손실 값: 939667840.0
저장 이미지:  style_transfer_result_at_iteration_2.png
2 번째 반복 완료: 160s
반복 횟수: 3
현재 손실 값: 646600400.0
저장 이미지:  style_transfer_result_at_iteration_3.png
3 번째 반복 완료: 153s
반복 횟수: 4
현재 손실 값: 499751460.0
저장 이미지:  style_transfer_result_at_iteration_4.png
4 번째 반복 완료: 170s
반복 횟수: 5
현재 손실 값: 419118700.0
저장 이미지:  style_transfer_result_at_iteration_5.png
5 번째 반복 완료: 178s
반복 횟수: 6
현재 손실 값: 333847500.0
저장 이미지:  style_transfer_result_at_iteration_6.png
6 번째 반복 완료: 176s
반복 횟수: 7
현재 손실 값: 297155100.0
저장 이미지:  style_transfer_result_at_iteration_7.png
7 번째 반복 완료: 160s
반복 횟수: 8
현재 손실 값: 271620500.0
저장 이미지:  style_transfer_result_at_iteration_8.png
8 번째 반복 완료: 138s
반복 횟수: 9
현재 손실 값: 251456720.0
저장 이미지:  style_transfer_result_at_iteration_9.png
9 번째 반복 완료: 143s
반복 횟수: 10
현재 손실 값: 232869980.0
저장 이미지

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# 콘텐츠 이미지
plt.imshow(load_img(target_image_path, target_size=(img_height, img_width)))
plt.figure()

# 스타일 이미지
plt.imshow(load_img(style_reference_image_path, target_size=(img_height, img_width)))
plt.figure()

# 생성된 이미지
plt.imshow(img)
plt.show()